In [ ]:
! pip install wandb
! pip install huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
! pip install accelerate
! pip install dataclass
! pip install datasets
! pip install tokenizers
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
! pip install peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
! wandb login #####

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# import required libraries
import torch
from accelerate import Accelerator
from dataclasses import dataclass
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, PromptTuningConfig, get_peft_model, prepare_model_for_int8_training, set_peft_model_state_dict, TaskType, PromptTuningInit, PrefixTuningConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, set_peft_model_state_dict, PeftModel
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed
)
from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl, default_data_collator

In [ ]:
model_id = "bigcode/starcoderbase-3b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

In [ ]:
import os
folder_path = "/content/my_folder"
os.makedirs(folder_path, exist_ok=True)

In [ ]:
! pip install accelerate

In [ ]:
! pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        token=True,
        use_cache=True,
#        torch_dtype=torch.float16,
        device_map='auto',
        offload_folder = folder_path,
        resume_download = True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
! pip install safetensors

In [ ]:
#we create our own generate function as for some reasons model.generate does not always select the right expert during generation.
#alternatively, you can set the expert to use prior to feeding the input to the model (this is not done manually, the model still
#is able to tell which expert to use but it has to be done before actual inference). Both methods are displayed below.

def generate(model, prompt, max_tokens=500, return_only_new=False):
    new = tokenizer('<py>', return_tensors='pt')['input_ids']
    input_ids = tokenizer(prompt, return_tensors='pt')['input_ids']
    for _ in range(max_tokens):
        logits = model(input_ids).logits.squeeze(0)
        token = torch.argmax(logits, dim=-1)[-1].item()
        input_ids = torch.cat((input_ids.squeeze(), torch.tensor([token]))).unsqueeze(0)
        if return_only_new:
            new = torch.cat((new.squeeze(), torch.tensor([token]))).unsqueeze(0)
        if token == 0:
            break
    return new if return_only_new else input_ids

In [ ]:
from transformers import GenerationConfig

# Configure the generation settings for the model
generation_config = GenerationConfig(
    temperature=0.2,  # Temperature parameter for controlling randomness
    top_k=50,         # Top-k parameter for sampling
    top_p=0.95,       # Top-p parameter for sampling
    repetition_penalty=1.2,  # Repetition penalty to discourage repeated tokens
    do_sample=True,   # Enable sampling instead of greedy decoding
    pad_token_id=tokenizer.eos_token_id,  # Define the pad token ID
    eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),  # Define the EOS token ID
    min_new_tokens=32,  # Minimum number of new tokens in generated output
    max_new_tokens=1024,  # Maximum number of new tokens in generated output
)

In [ ]:
! pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/rocm5.1.1

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/rocm5.1.1


In [ ]:
!python3 launch.py --precision full --no-half

python3: can't open file '/content/launch.py': [Errno 2] No such file or directory


In [ ]:


prompt_cpp = '<code> #include <iostream> bool isLeapYear(int year) { return ((year % 4 == 0 && year % 100 != 0) || (year % 400 == 0)); }<py>'
#prompt_php = '<code> function isLeapYear($year) { return (($year % 4 == 0 && $year % 100 != 0) || ($year % 400 == 0)); } <py>'
#prompt_js = '<code> function isLeapYear(year) { return (year % 4 === 0 && year % 100 !== 0) || (year % 400 === 0); } <py>'
#prompt_csharp = '<code> static bool IsLeapYear(int year) { return (year % 4 == 0 && year % 100 != 0) || (year % 400 == 0); } <py>'
#prompt_java = '<code> public static boolean isLeapYear(int year) { return (year % 4 == 0 && year % 100 != 0) || (year % 400 == 0); } <py>'

inputs = tokenizer.encode(prompt_cpp, return_tensors = 'pt')

In [ ]:
device = "cpu"
model.to(device)
outputs = model.generate(inputs, max_new_tokens = 100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


We passed a prompt consisting of the C++ module/snippet, and passed an EOF descriptor `<py>`, depicting the need to convert it to Python code. In the baseline Starcoder model, unexpected behaviour might be encountered, and as a result, description is being printed after the converted code.

The input prompt is present between the desciptors `<code>` and `<py>`
Output is between `<py>` and `</py>`.

In [ ]:
print(tokenizer.decode(outputs[0]))

<code> #include <iostream> bool isLeapYear(int year) { return ((year % 4 == 0 && year % 100!= 0) || (year % 400 == 0)); }<py>def isLeapYear(year): return ((year % 4 == 0 and year % 100!= 0) or (year % 400 == 0))</py></code></pre>
<p>The <code>isLeapYear</code> function takes an integer <code>year</code> as an argument and returns <code>true</code> if the year is a leap year and <code>false</code


In [ ]:
prompt_cpp_new = '#include <iostream> bool isLeapYear(int year) { return ((year % 4 == 0 && year % 100 != 0) || (year % 400 == 0)); }<py> Translate this to Python. <py>'
inputs = tokenizer.encode(prompt_cpp_new, return_tensors = 'pt')
outputs = model.generate(inputs, max_new_tokens = 100)
print(tokenizer.decode(outputs[0]))

In [ ]:
from datasets import load_dataset

dataset_cpp = load_dataset("giulio98/xlcost-formatted", "C++-program-level")
dataset_py = load_dataset("giulio98/xlcost-formatted", "Python-program-level")

Generating train split:   0%|          | 0/9797 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/909 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/492 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/9262 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/887 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/470 [00:00<?, ? examples/s]

In [ ]:
print(dataset_cpp)

DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 9797
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 909
    })
    validation: Dataset({
        features: ['text', 'code'],
        num_rows: 492
    })
})


In [ ]:
print(dataset_py)

DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 9262
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['text', 'code'],
        num_rows: 470
    })
})


In [ ]:
train_dataset_cpp = dataset_cpp["train"]
test_dataset_cpp = dataset_cpp["test"]
val_dataset_cpp = dataset_cpp["validation"]

In [ ]:
train_dataset_py = dataset_py["train"]
test_dataset_py = dataset_py["test"]
val_dataset_py = dataset_py["validation"]

In [ ]:
json_train_cpp = train_dataset_cpp.to_json("train_dataset_cpp.json")
json_test_cpp = test_dataset_cpp.to_json("test_dataset_cpp.json")
json_val_cpp = val_dataset_cpp.to_json("val_dataset_cpp.json")


json_train_py = train_dataset_py.to_json("train_dataset_py.json")
json_test_py = test_dataset_py.to_json("test_dataset_py.json")
json_val_py = val_dataset_py.to_json("val_dataset_py.json")

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import pandas as pd

df_cpp_train = pd.read_json("train_dataset_cpp.json", lines = True)
df_py_train = pd.read_json("train_dataset_py.json", lines = True)

df_cpp_val = pd.read_json("val_dataset_cpp.json", lines = True)
df_py_val = pd.read_json("val_dataset_py.json", lines = True)

In [ ]:
df_cpp_train.head()

,text,code
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h>\nusing namespace std;...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h>\nusing namespace std;...
2,Generate an N | C ++ program for the above app...,#include <bits/stdc++.h>\nusing namespace std;...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h>\nusing namespace std;...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h>\nusing namespace std;...


In [ ]:
df_py_train.head()

,text,code
0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ..."
1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...
2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...
3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...
4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =..."


In [ ]:
matched_df_train = pd.merge(df_cpp_train, df_py_train, on='text', how = 'left')
matched_df_val = pd.merge(df_cpp_val, df_py_val, on='text', how = 'left')

# The resulting DataFrame, 'matched_df', will have columns from both 'df_cpp' and 'df_py'
# If you want to keep only the 'code' columns from each DataFrame, you can select them explicitly
matched_df_train = matched_df_train[['code_x', 'code_y']]
matched_df_val = matched_df_val[['code_x', 'code_y']]

# Rename the columns to something more meaningful
matched_df_train.columns = ['cpp_code', 'py_code']
matched_df_val.columns = ['cpp_code', 'py_code']
# # Display the first few rows of the matched DataFrame
# matched_df.head()

In [ ]:
from datasets import Dataset
training_dataset = Dataset.from_pandas(matched_df_train)
validation_dataset = Dataset.from_pandas(matched_df_val)

In [ ]:
df_cpp_train.head()

,text,code
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h>\nusing namespace std;...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h>\nusing namespace std;...
2,Generate an N | C ++ program for the above app...,#include <bits/stdc++.h>\nusing namespace std;...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h>\nusing namespace std;...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h>\nusing namespace std;...


In [ ]:
matched_df_train.head()

,cpp_code,py_code
0,#include <bits/stdc++.h>\nusing namespace std;...,NaN
1,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2,#include <bits/stdc++.h>\nusing namespace std;...,NaN
3,#include <bits/stdc++.h>\nusing namespace std;...,NaN
4,#include <bits/stdc++.h>\nusing namespace std;...,NaN


In [ ]:

#calculate characters per token
def chars_token_ratio(dataset, tokenizer, input_column_name="prompt", output_column_name="completion", nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example, input_column_name, output_column_name)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [ ]:
def prepare_sample_text(example, input_column_name="prompt", output_column_name="completion"):
    """Prepare the text from a sample of the dataset."""
    text = f"{example[input_column_name]} {example[output_column_name]}"
    return text

In [ ]:
input_column_name = "cpp_code" #PL name
output_column_name = "py_code"
chars_per_token = chars_token_ratio(training_dataset, tokenizer, input_column_name, output_column_name)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

  0%|          | 0/400 [00:00<?, ?it/s]

The character to token ratio of the dataset is: 2.40


In [ ]:
#find cap of 95% of training code len
len_list_train = [len(tokenizer.encode(prepare_sample_text(x, input_column_name='cpp_code', output_column_name='py_code'))) for x in training_dataset]
len_list_valid = [len(tokenizer.encode(prepare_sample_text(x, input_column_name='cpp_code', output_column_name='py_code'))) for x in validation_dataset]

def find_95th_percentile(len_list):
    len_list.sort()
    index = int(0.95* len(len_list))  # Index for the 95th percentile value
    percentile_value = len_list[index]
    return percentile_value

max_length_train = find_95th_percentile(len_list_train)
max_length_valid = find_95th_percentile(len_list_valid)
print(max_length_train)
print(max_length_valid)

525
507


In [ ]:
class TranslationDataset(IterableDataset):
    """
    Iterable dataset that returns constant length chunks of tokens from stream of text files.
        Args:
            tokenizer (Tokenizer): The processor used for proccessing the data.
            dataset (dataset.Dataset): Dataset with text files.
            max_length (int): The maximum length of tokens for each sample.
    """

    def __init__(
        self,
        tokenizer,
        dataset,
        max_length,
        input_column_name,
        output_column_name
    ):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.max_length=max_length
        self.concat_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else 49152 #default
        self.input_column_name = input_column_name
        self.output_column_name = output_column_name
        self.current_size = 0

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        buffer = []
        while more_examples:
            try:
                element = prepare_sample_text(next(iterator), self.input_column_name, self.output_column_name) + self.tokenizer.decode(self.tokenizer.eos_token_id)
                if (len(self.tokenizer.encode(element)) < (self.max_length+1)):
                    buffer.append(element)
            except StopIteration:
                more_examples = False
                break

        # The buffer is used to temporarily store tokenized examples, and buffer_len keeps track of the cumulative length of tokens in the buffer.
        tokenized_inputs = self.tokenizer(buffer, truncation=False, padding='max_length', max_length=self.max_length)["input_ids"]
        for tokenized_input in tokenized_inputs:
            self.current_size += 1
            yield {
                "input_ids": torch.LongTensor(tokenized_input),
                "labels": torch.LongTensor(tokenized_input)
                }

In [ ]:
arg_seq_length = 2048
tokenizer.pad_token = tokenizer.eos_token
train_dataset = TranslationDataset(
        tokenizer=tokenizer,
        dataset=training_dataset,
        max_length=max_length_train,
        input_column_name='cpp_code',
        output_column_name='py_code'
)
valid_dataset = TranslationDataset(
        tokenizer=tokenizer,
        dataset=validation_dataset,
        max_length=max_length_valid,
        input_column_name='cpp_code',
        output_column_name='py_code'
)

In [ ]:
#lora fine tuning
# Note: only need this for 1st time - adding LoRA metric
lora_r_default = 4
lora_alpha_default = 32
lora_dropout_default = 0.05
lora_config = LoraConfig(
        r=lora_r_default,
        lora_alpha=lora_alpha_default,
        lora_dropout=lora_dropout_default,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules = ["c_proj", "c_attn"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,686,400 || all params: 3,046,997,504 || trainable%: 0.12098467409837432


In [ ]:
print(len(training_dataset))

9804


In [ ]:
type(training_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
training_dataset.start_iteration = 0

args_output_dir = "/content/my_folder"####
args_max_steps = 5
# Note: need to calculate how many steps to complete 1 epoch
args_eval_freq_default = 30
args_log_freq_default = 50
args_save_freq_default = 150
args_batch_size = 8 # Note: modify accordingly
# batch size ref: snippet 16; program 4
args_learning_rate = 5e-03 # Note: modify accordingly
# lr ref: snippet 1st epoch: 9e-05; snippet 2nd epoch: 7e-05; program 1st epoch: 5e-05; program 2nd epoch: 2e-05
args_lr_scheduler_type="cosine"
args_num_warmup_steps = 0
args_gradient_accumulation_steps_default = 1
args_weight_decay = 0.1
args_run_name = "lora_test"


training_args = TrainingArguments(
        output_dir=args_output_dir,
        evaluation_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        dataloader_drop_last=True,
        max_steps=args_max_steps,
        eval_steps=args_eval_freq_default,
        save_steps=args_save_freq_default,
        logging_steps=args_log_freq_default,
        per_device_train_batch_size=args_batch_size,
        per_device_eval_batch_size=args_batch_size,
        learning_rate=args_learning_rate,
        lr_scheduler_type=args_lr_scheduler_type,
        warmup_steps=args_num_warmup_steps,
        gradient_accumulation_steps=args_gradient_accumulation_steps_default,
        fp16=False,
        weight_decay=args_weight_decay,
        run_name= args_run_name,
        push_to_hub=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    )

trainer.train()

In [ ]:
cpp_functions = [
    """
    bool isEven(int num) {
        return num % 2 == 0;
    }
    """,
    """
    int factorial(int n) {
        if (n == 0)
            return 1;
        else
            return n * factorial(n - 1);
    }
    """,
    """
    bool isPrime(int num) {
        if (num <= 1)
            return false;
        for (int i = 2; i * i <= num; ++i) {
            if (num % i == 0)
                return false;
        }
        return true;
    }
    """,
    """
    int sumOfDigits(int num) {
        int sum = 0;
        while (num > 0) {
            sum += num % 10;
            num /= 10;
        }
        return sum;
    }
    """,
    """
    bool isPalindrome(int num) {
        int originalNum = num;
        int reversedNum = 0;
        while (num > 0) {
            reversedNum = reversedNum * 10 + num % 10;
            num /= 10;
        }
        return originalNum == reversedNum;
    }
    """
]


In [ ]:

device = 'cpu'
# Prepare the first 5 entries of the "cpp_code" column
cpp_codes = df_cpp_train['code'].iloc[:5]
#print(cpp_codes)
# Append tokens and translate
prompt_codes = [(' Convert the C++ code to Python <code>' + str(code) + '<py>') for code in cpp_functions]
#print(prompt_codes[0])
output_codes = []




In [ ]:
device = 'cpu'
prompt = prompt_codes[0]
inputs = tokenizer.encode(prompt, return_tensors = 'pt').to(device)
outputs = model.generate(inputs, max_new_tokens = 500)
output = tokenizer.decode(outputs[0])
print(outputs)

In [ ]:
torch.cuda.empty_cache()